# Info
元のコード : https://github.com/jwkirchenbauer/lm-watermarking

参考説明(kr) : https://colab.research.google.com/drive/1V8eMbM6LZNOKADDZZFoXQ2ukdGQ4dkbl?usp=drive_link

## Watermark AI-Generated Texts

- 大規模言語モデル（LLM）のウォーターマーキングおよび検出アルゴリズム

- AI生成テキストの出力を追跡し、真実を確認するためのウォーターマークの追加技術。

### A Watermark for Large Language Models

https://github.com/jwkirchenbauer/lm-watermarking

In [1]:
# Transformerモデルを使用してaccelerateをインストール
!pip install -q accelerate
# コードのクローン
!git clone -q https://github.com/jwkirchenbauer/lm-watermarking.git lm_watermarking
# 確認
%cd lm_watermarking

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 4.6 MB/s eta 0:00:00
/content/lm_watermarking


## LLM ロード
https://huggingface.co/Qwen/Qwen2-1.5B-Instruct

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

# Alibabaによって作成されたQwen2モデルを使用
# AutoModelForCausalLM.from_pretrained（）：Hugging FaceのTransformersライブラリが提供する関数
# モデルの読み込み関数
# device_map：CPUまたはGPUに割り当てるかどうかを決定する
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
# AutoTokenizer.from_pretrained（）：Hugging FaceのTransformersライブラリが提供する関数
# 文章をトークンに分割し、各トークンに対してインデックスを割り当てます。
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## サンプルテキストの生成

In [14]:
import huggingface_hub
huggingface_hub.login()

In [19]:
prompt = "Large Language Modelについて簡単に説明してください。"
messages = [
    {"role": "system", "content": "あなたは親切なチャットボットです。"},
    {"role": "user", "content": prompt}
]
# tokenizer.apply_chat_template（）：会話スクリプトを処理し、特定の形式に変換するメソッドです。
# tokenize=Falseはトークン化しないようにするオプションです。
# add_generation_prompt=Trueは生成プロンプトを追加するオプションです。
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# tokenizer（[text]、return_tensors = "pt"）：変換されたテキストをモデルが理解できる形式にトークン化し、テンソルに変換するメソッドです。
#"pt"はPyTorchテンソルを意味します。
#.to（device）は、データを指定されたデバイス（例：GPU）に移動するメソッドです。
model_inputs = tokenizer([text],return_tensors = "pt").to(device)

# model.generate（）：モデルに入力データを提供して新しいテキストを生成するメソッドです。
# model_inputs.input_idsはモデルの入力トークンIDを意味します
# max_new_tokens = 128は生成する最大トークン数を制限するオプションです。
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128
)

# 生成された出力から入力トークンIDを除外した部分を抽出するリストコンプリヘンションです。
# これにより、モデルが生成した新しいトークンを取得します。
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# tokenizer.batch_decode（）：生成されたトークンIDをテキストにデコードするメソッドです。
# skip_special_tokens=Trueは特殊トークンをスキップするように設定されたオプションです。
# [0]はバッチから最初の結果のみを取得するようにします。
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

大規模言語モデル（Large Language Model、LLM）とは、自然言語処理や理解に優れたAIモデルの一つで、一般的にはコンピューターサイエンスの分野における主な研究課題である。LMLは、単語や文の意味を理解し、それに基づいて新たな情報を生成する能力を持っています。

この能力は、例えばテキスト分析、文章作成、リクエスト応答、情報検索、予測分析など、広範囲にわたるアプリケーションに役立ちます


## 検証_非Watermarking

In [20]:
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
# WatermarkDetector()
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting.
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0, #watermark
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

# z_scoreとprediction確認
print(score_dict)

{'num_tokens_scored': 125, 'num_green_tokens': 16, 'green_fraction': 0.128, 'z_score': -3.150026454915366, 'p_value': 0.9991837216059614, 'z_score_at_T': tensor([-0.5774, -0.8165,  0.3333,  0.0000, -0.2582, -0.4714, -0.6547,  0.0000,
        -0.1925,  0.3651,  0.1741,  0.0000, -0.1601, -0.3086, -0.4472, -0.5774,
        -0.1400, -0.2722,  0.1325,  0.5164,  0.3780,  0.2462,  0.1204,  0.4714,
         0.3464,  0.2265,  0.1111,  0.0000, -0.1072, -0.2108, -0.3111, -0.4082,
        -0.1005, -0.1980,  0.0976,  0.0000, -0.0949, -0.1873, -0.2774, -0.3651,
        -0.4508, -0.5345, -0.6163, -0.6963, -0.7746, -0.8513, -0.9264, -0.6667,
        -0.7423, -0.8165, -0.8893, -0.9608, -1.0310, -1.0999, -1.1677, -1.2344,
        -1.3000, -1.3646, -1.4281, -1.4907, -1.5524, -1.6131, -1.6730, -1.7321,
        -1.7903, -1.8477, -1.9044, -1.9604, -2.0156, -2.0702, -2.1241, -2.1773,
        -2.2299, -2.0135, -2.0667, -2.1193, -2.1712, -2.2226, -2.2735, -2.3238,
        -2.3736, -2.4228, -2.4715, -2.5198, -2

## watermark

In [21]:
from transformers import LogitsProcessorList
#  WatermarkLogitsProcessor(): 単語を再配列化
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="selfhash") # equivalent to `ff-anchored_minhash_prf-4-True-15485863`

In [22]:
prompt = "Large Language Modelについて簡単に説明してください。"
messages = [
    {"role": "system", "content": "あなたは親切なチャットボットです。"},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128,

    logits_processor=LogitsProcessorList([watermark_processor])
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

大規模言語モデル（LGM）とは、人工知能技術が進化した結果、テキスト理解や生成、リズムの理解といった複雑なコミュニケーションツールを提供するための高度なAIプラットフォームのことです。

具体的には、AIの学習プロセスを通じて大量のデータと情報を処理し、それを理解し表現することで、人間の思考とコミュニケーションの形を再現することが可能です。LGMは、さまざまな場面で利用でき、文書の翻訳、情報の提供、予測型推論など、


In [ ]:
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 125, 'num_green_tokens': 62, 'green_fraction': 0.496, 'z_score': 6.351692687780163, 'p_value': 1.0647922005016939e-10, 'z_score_at_T': tensor([-0.5774,  0.8165,  1.6667,  1.1547,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925,  0.3651,  0.8704,  0.6667,  1.1209,  0.9258,  0.7454,  0.5774,
         0.9802,  0.8165,  1.1921,  1.5492,  1.8898,  2.2156,  2.0466,  1.8856,
         2.1939,  2.0381,  2.3333,  2.1822,  2.4659,  2.7406,  2.5924,  2.4495,
         2.3116,  2.5744,  2.8301,  3.0792,  2.9424,  2.8098,  2.6811,  2.5560,
         2.7952,  2.6726,  2.9055,  3.1334,  3.0123,  3.2348,  3.4528,  3.3333,
         3.5466,  3.7559,  3.6380,  3.5228,  3.7273,  3.9284,  3.8146,  4.0119,
         4.2060,  4.3970,  4.2844,  4.1740,  4.3614,  4.5461,  4.7281,  4.9075,
         4.7980,  4.6904,  4.8669,  4.7610,  4.9348,  5.1065,  5.2760,  5.4433,
         5.6086,  5.5035,  5.6667,  5.5630,  5.7242,  5.8835,  6.0410,  6.1968,
         6.0943,  5.9932,  5.8936,  5.7955,

## 文章の一部だけ入れても良いか

In [ ]:
response = """大規模言語モデル（Large Language Model、LLM）とは、自然言語処理や理解に優れたAIモデルの一つで、一般的にはコンピューターサイエンスの分野における主な研究課題である。LMLは、単語や文の意味を理解し、それに基づいて新たな情報を生成する能力を持っています。
"""

watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 80, 'num_green_tokens': 21, 'green_fraction': 0.2625, 'z_score': 0.2581988897471611, 'p_value': 0.3981267073688196, 'z_score_at_T': tensor([-0.5774,  0.8165,  0.3333,  0.0000,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925, -0.3651, -0.5222, -0.6667, -0.1601, -0.3086, -0.4472, -0.5774,
        -0.7001, -0.8165, -0.9272, -1.0328, -1.1339, -1.2309, -1.3242, -1.4142,
        -1.0392, -0.6794, -0.7778, -0.4364, -0.5361, -0.6325, -0.7259, -0.8165,
        -0.9045, -0.9901, -0.6831, -0.7698, -0.8542, -0.9366, -0.6472, -0.7303,
        -0.8115, -0.8909, -0.9685, -1.0445, -0.7746, -0.7746, -0.8513, -0.5895,
        -0.3333, -0.4124, -0.4899, -0.2425, -0.3203, -0.3965, -0.4714, -0.5449,
        -0.3086, -0.3824, -0.4549, -0.2255, -0.2981, -0.3696, -0.1466, -0.2182,
        -0.2887, -0.3581, -0.4264, -0.4937, -0.2801, -0.3475, -0.4140, -0.4140,
        -0.4140, -0.4140, -0.2056,  0.0000,  0.2027,  0.4027,  0.3333,  0.5298,
         0.4606,  0.3922,  0.3248,  0.2582]), 

In [ ]:
response = """大規模言語モデル（Large Language Model、LLM）とは、自然言語処理や理解に優れたAIモデルの一つで"""

watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 74, 'num_green_tokens': 20, 'green_fraction': 0.2702702702702703, 'z_score': 0.4026936331284146, 'p_value': 0.34358680915482404, 'z_score_at_T': tensor([-0.5774,  0.8165,  0.3333,  0.0000,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925, -0.3651, -0.5222, -0.6667, -0.1601, -0.3086, -0.4472, -0.5774,
        -0.7001, -0.8165, -0.9272, -1.0328, -1.1339, -1.2309, -1.3242, -1.4142,
        -1.0392, -0.6794, -0.7778, -0.4364, -0.5361, -0.6325, -0.7259, -0.8165,
        -0.9045, -0.9901, -0.6831, -0.7698, -0.8542, -0.9366, -0.6472, -0.7303,
        -0.8115, -0.8909, -0.9685, -1.0445, -0.7746, -0.7746, -0.8513, -0.5895,
        -0.3333, -0.4124, -0.4899, -0.2425, -0.3203, -0.3965, -0.4714, -0.5449,
        -0.3086, -0.3824, -0.4549, -0.2255, -0.2981, -0.3696, -0.1466, -0.2182,
        -0.2887, -0.3581, -0.4264, -0.4937, -0.2801, -0.3475, -0.4140, -0.4140,
        -0.4140, -0.4140, -0.2056,  0.0000,  0.2027,  0.4027]), 'prediction': False}


# Result
文字列を削除するほど、watermarkの推測が難しくなる

# watermaking回避

1.   ChatGPTで生成されたテキストを異なる形で再構成することにより、コンテンツに適用されたウォーターマークを無効にすることができます。


2.  生成されたテキストを再度作成する補助AIモデルを使用する方法です。この方法により、ウォーターマークのシーケンスが壊れるため、AI生成コンテンツとして検出することが難しくなります。

# ChatGPTが作成した文章かどうわかるか？

ChatGPTで作成されたテキストであるかどうかを判断するのは難しいかもしれませんが、AI検出ツールはいくつかのパターンを認識するのに役立ちます。

あなたが今読んでいるコンテンツがChatGPTで作成されたものであることを示す8つの特徴は以下の通りです。

##1. パターンの不一致を見つける
   前述のように、AIが生成したテキストには多くの認識可能なパターンがあります。

   これにはテキストの文法や構造のパターンだけでなく、テキスト自体がどれだけ予測可能かというパターンも含まれます。

   AIは枠組みから外れた考えを持つことができず、むしろ安全で一貫した結果を提供し、読みやすく理解しやすいテキストを目指しています。

   過度に単純で予測可能な言語はAIが生成したテキストの特徴です。

##2. 人間のエラーの兆候を確認する
   知らないかもしれませんが、人間が書いたテキストには多くの誤りが含まれています。

   これは非常に普通のことであり、何世紀にもわたって続いてきた慣行です。

   編集者がいると人間のテキストでは誤って配置された句読点や文法、時制の問題などが見落とされることがあります。

   しかし、AIテキストに比べて人間のテキストはより洗練され、自然です。

   ChatGPTや他のAIツールはテキストを生成するのではなく、学習したすべてのデータからテキストを生成します。

   ここには私たちにとって不自然に聞こえるかもしれないが、AIツール自体には識別できない文法や語の選択の誤りが含まれる可能性があります。

##3. 説明言語が不足しているかどうかを見る
   AIが生成したコンテンツの難解さのスコアが直接的な要因です。

   大多数のLLMはユーザーが混乱しないように設計およびプログラムされています。そのため、これらのツールはよりシンプルで理解しやすい言語を使用します。

   また、単語の選択においても多様性が不足しており、同じ単語やフレーズが同じテキスト内で何度も現れることが多いため、それを見分けることができます。

##4. コンテキストに注意する
   コンテキストに注意する理由は何でしょうか？ AIツールは多くの場合、正確なコンテキストを提供できないからです。

   AIは基本的に以前のデータセットからの出力と応答を生成することを覚えておいてください。

   ChatGPTが毎回新しい会話の文脈を持続的に理解することは簡単ではありません。

   ChatGPTはますます賢くなり、以前の会話の文脈を理解し学習することができます。

   ただし、テキストの出力に関しては、ChatGPTや他のAIツールは依然として人間の作成者が容易に含めることができる文脈を提供することがかなり不十分です。

##5. 過剰な遷移語の使用
   遷移語は文とアイデアを結びつけるのに優れています。

   この構造は人間の作家にとって自然なものですが、AIツールの場合、過剰に遷移語を使用して補完されるようです。

   このようなLLMは類似した文構造と長さを提供するように学習されています。

   これを防ぐために、文や概念を不自然な方法で結ぼうとする過剰なアルゴリズムもあるようです。

   テキストを読んで奇妙に聞こえた場合、そのテキストの制作に人工知能が関与していた可能性が高いです。

##6. 正しそうでも意味が通らない文
   一見すると正しいようでも、文を理解しようとすると少しズレたものを見たことがあれば、AIが作成した可能性が高いです。

   これらのツールは文の構造に関してはよく訓練されていますが、文の主題に関してはしばしば見逃されることがあります。

   ChatGPTや他のAIツールはどこかで学んだデータを繰り返すだけなので、不自然なフレーズや文が出てくる可能性があることを常に覚えておいてください。

##7. 独創性の欠如
   大多数の人工知能テキストを読んでいると、独創性がまったく感じられません。

   人間の作家のニュアンスと精神が欠けています。 GoogleのEEATガイドラインにより、AIが生成したコンテンツが高いランキングを獲得するのは難しい理由があります。

   Googleはコンテンツでの経験、専門知識、権威性、信頼性を示す事例を探しています。

   これらすべては人工知能が提供するのが難しいものであり、特に独創的で考えを刺激するような方法で提供するのが難しいものです。

   さらに、完全に異なる出典からのテキストを完全に盗用したAIテキストの例もありました。

   AIツールは非常に役立つものですが、人間に比べて独創性が不足しています。

##8. 事実の誤り
   既にAI幻覚について簡単に見てきましたが、残念ながらAIの文書作成ツールとしての事業を行うことができます。

   ChatGPTが実際に不正確な出力を提供する場合、その真偽と正確性を確認することは使用者の責任です。

   ChatGPT自体は眼一つくらいちぎりましたが、事実の誤りを提供することがあります。

   テキストの特定の内容が明らかに間違っていることがわかった場合、AIツールまたは恐ろしく間違った情報を

# 生成監査Tool

- SEO.ai：無料のAI検知ツールも提供しています。このツールは、最新のGPT-3.5やGPT-4など、他のLLMも検知できます。

- Copyleaks

- GPTZero：ChatGPTや他のLLMによって作成されたコンテンツを検知するためのAIツールです。